In [178]:
import pandas as pd
import os
import sklearn
import numpy as np
import pandas as pd
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


cleaning of data

In [179]:
fifa_21=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ai/players_21.csv")

In [180]:
missing_values = fifa_21.isna().sum() / fifa_21.shape[0] * 100
columns_to_drop = missing_values[missing_values > 30].index.tolist()
fifa_21 = fifa_21.drop(columns_to_drop, axis=1)
fifa_21.to_csv('dataset_with_missing_values_removed.csv', index=False)
fifa_21





,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,62+3,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,61+3,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,62+3,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,75+3,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,https://sofifa.com/player/257710/mengxuan-zhan...,Zhang Mengxuan,张梦炫,CB,47,52,70000.0,1000.0,21,...,45+2,47+2,47+2,47+2,45+2,15+2,https://cdn.sofifa.net/players/257/710/21_120.png,https://cdn.sofifa.net/teams/112165/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/flags/cn.png
18940,257933,https://sofifa.com/player/257933/wenzhuo-huang...,Huang Wenzhou,黄文卓,CM,47,53,70000.0,1000.0,21,...,47+2,46+2,46+2,46+2,47+2,15+2,https://cdn.sofifa.net/players/257/933/21_120.png,https://cdn.sofifa.net/teams/112540/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/flags/cn.png
18941,257936,https://sofifa.com/player/257936/yue-song/210002,Song Yue,宋岳,CM,47,47,45000.0,2000.0,28,...,47,46+1,46+1,46+1,47,11+2,https://cdn.sofifa.net/players/257/936/21_120.png,https://cdn.sofifa.net/teams/111774/60.png,https://cdn.sofifa.net/flags/cn.png,https://cdn.sofifa.net/flags/cn.png
18942,258736,https://sofifa.com/player/258736/vani-da-silva...,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,47,67,130000.0,500.0,17,...,33+2,26+2,26+2,26+2,33+2,14+2,https://cdn.sofifa.net/players/258/736/21_120.png,https://cdn.sofifa.net/teams/1920/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/flags/gb-eng.png


In [181]:
correlation_matrix = fifa_21.corr()

<ipython-input-181-4f3a38e95ff5>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = fifa_21.corr()


feature selection

In [182]:
selected_features= correlation_matrix['overall'].sort_values(ascending=False).index
selected_features[:10]


Index(['overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'release_clause_eur', 'wage_eur', 'value_eur',
       'power_shot_power'],
      dtype='object')

In [183]:
selected_features=[ 'overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'release_clause_eur', 'wage_eur', 'value_eur',
       'power_shot_power']

In [184]:
new_df = selected_features.copy()
new_fifa=pd.DataFrame(fifa_21[new_df])
new_fifa

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,93,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91
...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50


In [185]:


new_fifa = new_fifa.fillna(new_fifa.mean())
new_fifa




,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,93,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91
...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50


In [186]:
new_fifa

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,93,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91
...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50


In [187]:
row_index = 0
new_fifa.loc[row_index, 'overall'] = 92
new_fifa.to_csv('updated_dataset.csv', index=False)
new_fifa


,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,92,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91
...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50


In [188]:
data=new_fifa.copy()

In [189]:
data

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,92,94,91.0,96,95.0,93,138400000.0,560000.0,103500000.0,86
1,92,95,81.0,95,89.0,92,75900000.0,220000.0,63000000.0,94
2,91,93,78.0,88,85.0,91,132000000.0,240000.0,111000000.0,89
3,91,91,86.0,93,94.0,91,166500000.0,270000.0,132000000.0,80
4,91,91,93.0,91,88.0,91,161000000.0,370000.0,129000000.0,91
...,...,...,...,...,...,...,...,...,...,...
18939,47,48,26.0,40,27.0,52,57000.0,1000.0,70000.0,35
18940,47,50,49.0,35,47.0,53,72000.0,1000.0,70000.0,38
18941,47,44,49.0,35,46.0,47,47000.0,2000.0,45000.0,49
18942,47,53,40.0,45,53.0,67,165000.0,500.0,130000.0,50


training and testing

In [190]:
y=new_fifa['overall']


In [191]:
X=new_fifa[[ 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'release_clause_eur', 'wage_eur', 'value_eur',
       'power_shot_power']]


In [192]:
sc=StandardScaler()
scaled=sc.fit_transform(X)
X=pd.DataFrame(scaled, columns=X.columns)

In [193]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [194]:
from sklearn.ensemble import RandomForestRegressor


In [195]:
rf = RandomForestRegressor()
rf.fit(Xtrain, Ytrain)



RandomForestRegressor()

In [196]:
from sklearn.metrics import accuracy_score

In [197]:
y_pred=rf.predict(Xtest)
y_pred

array([65.73, 64.86, 76.73, ..., 73.97, 60.11, 62.29])

In [198]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [199]:
mean_absolute_error(y_pred,Ytest)

0.6111375032990236

In [200]:
from sklearn.tree import DecisionTreeRegressor


In [201]:
dt = DecisionTreeRegressor()
dt.fit(Xtrain, Ytrain)

DecisionTreeRegressor()

In [202]:

y_pred = dt.predict(Xtest)
y_pred



array([65., 63., 77., ..., 74., 59., 63.])

In [203]:
mae=mean_squared_error(y_pred,Ytest)
mae

2.602797571918712

In [204]:
from xgboost import XGBRegressor


In [205]:
xgb = XGBRegressor()


In [206]:
xgb.fit(Xtrain, Ytrain)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [207]:
y_pred = xgb.predict(Xtest)


In [208]:
y_pred

array([65.39276 , 64.19836 , 76.7409  , ..., 73.96726 , 60.090412,
       62.07048 ], dtype=float32)

In [209]:
mse = mean_absolute_error(Ytest, y_pred)
mse

0.732725938287062

optimization and fine tuning

In [210]:
from sklearn.model_selection import RandomizedSearchCV
param_distributions = {
    'n_estimators': np.arange(10, 50, step=10),
    'max_depth': np.arange(2, 12, step=1)
}
n_iter = 100
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search.fit(Xtrain, Ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 40 is smaller than n_iter=100. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   param_distributions={'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
                                        'n_estimators': array([10, 20, 30, 40])})

In [211]:
best_params = random_search.best_params_
best_rf = RandomForestRegressor(**best_params)
best_rf.fit(Xtrain, Ytrain)

RandomForestRegressor(max_depth=11, n_estimators=30)

In [212]:
y_pred = best_rf.predict(Xtest)
mse = mean_squared_error(Ytest, y_pred)
mse

1.27519329674029

In [213]:
param_distributions = {
    'max_depth': np.arange(2, 10, step=1),
    'min_samples_split': np.arange(2, 100, step=10),
    'max_features': np.arange(1, 10, step=1)
}
n_iter = 100
random_search = RandomizedSearchCV(
    estimator=DecisionTreeRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search.fit(Xtrain, Ytrain)


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=100,
                   param_distributions={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'max_features': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                                        'min_samples_split': array([ 2, 12, 22, 32, 42, 52, 62, 72, 82, 92])})

In [214]:
best_params = random_search.best_params_
best_dt = DecisionTreeRegressor(**best_params)
best_dt.fit(Xtrain, Ytrain)

DecisionTreeRegressor(max_depth=9, max_features=9)

In [215]:
y_pred = best_dt.predict(Xtest)
mean_squared_error(Ytest, y_pred)

2.207136286670671

In [216]:
param_distributions = {
    'n_estimators': np.arange(100, 500, step=100),
    'max_depth': np.arange(2, 10, step=1),
    'learning_rate': np.arange(0.01, 0.1, step=0.01),
    'gamma': np.arange(0, 1, step=0.1),
    'subsample': np.arange(0.5, 1, step=0.1),
    'colsample_bytree': np.arange(0.5, 1, step=0.1)
}
n_iter = 100
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search.fit(Xtrain, Ytrain)





RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=100,
                   param_distributions={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'gamma': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]),
                                        'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'n_estimators': array([100, 200, 300, 400]),
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])})

In [217]:
best_params = random_search.best_params_
best_xgb = XGBRegressor(**best_params)
best_xgb.fit(Xtrain, Ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8999999999999999, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.7000000000000001,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.04, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=400, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [218]:
y_pred = best_xgb.predict(Xtest)
mean_squared_error(Ytest, y_pred)


0.9740799616331365

In [219]:
#COMPARING WITH FIFA 2022
fifa_22=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ai/players_22.csv")
fifa_22

<ipython-input-219-ef8ba59be450>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  fifa_22=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ai/players_22.csv")


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png


In [220]:
correlation_matrix = fifa_22.corr()

<ipython-input-220-c5df40832efd>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = fifa_22.corr()


In [221]:
features_22= correlation_matrix['overall'].sort_values(ascending=False).index
features_22[:10]

Index(['overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'wage_eur', 'power_shot_power', 'value_eur',
       'release_clause_eur'],
      dtype='object')

In [222]:
features_22=['overall', 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'release_clause_eur', 'wage_eur', 'value_eur',
       'power_shot_power']

In [223]:
df_22 = features_22.copy()
newff_22=pd.DataFrame(fifa_22[df_22])
newff_22

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,93,94,91.0,96,95.0,93,144300000.0,320000.0,78000000.0,86
1,92,93,79.0,88,86.0,92,197200000.0,270000.0,119500000.0,90
2,91,94,80.0,95,88.0,91,83300000.0,270000.0,45000000.0,94
3,91,89,86.0,93,94.0,91,238700000.0,270000.0,129000000.0,80
4,91,91,93.0,89,88.0,91,232200000.0,350000.0,125500000.0,91
...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,114000.0,1000.0,70000.0,46
19235,47,49,50.0,47,46.0,59,193000.0,500.0,110000.0,49
19236,47,46,45.0,36,49.0,55,175000.0,500.0,100000.0,50
19237,47,48,36.0,47,48.0,60,239000.0,500.0,110000.0,48


In [224]:
newff_22 = newff_22.fillna(newff_22.mean())
newff_22

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,93,94,91.0,96,95.0,93,144300000.0,320000.0,78000000.0,86
1,92,93,79.0,88,86.0,92,197200000.0,270000.0,119500000.0,90
2,91,94,80.0,95,88.0,91,83300000.0,270000.0,45000000.0,94
3,91,89,86.0,93,94.0,91,238700000.0,270000.0,129000000.0,80
4,91,91,93.0,89,88.0,91,232200000.0,350000.0,125500000.0,91
...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,114000.0,1000.0,70000.0,46
19235,47,49,50.0,47,46.0,59,193000.0,500.0,110000.0,49
19236,47,46,45.0,36,49.0,55,175000.0,500.0,100000.0,50
19237,47,48,36.0,47,48.0,60,239000.0,500.0,110000.0,48


In [225]:
row_index = 0
newff_22.loc[row_index, 'overall'] = 92
newff_22.to_csv('updated_dataset.csv', index=False)
newff_22


,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,92,94,91.0,96,95.0,93,144300000.0,320000.0,78000000.0,86
1,92,93,79.0,88,86.0,92,197200000.0,270000.0,119500000.0,90
2,91,94,80.0,95,88.0,91,83300000.0,270000.0,45000000.0,94
3,91,89,86.0,93,94.0,91,238700000.0,270000.0,129000000.0,80
4,91,91,93.0,89,88.0,91,232200000.0,350000.0,125500000.0,91
...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,114000.0,1000.0,70000.0,46
19235,47,49,50.0,47,46.0,59,193000.0,500.0,110000.0,49
19236,47,46,45.0,36,49.0,55,175000.0,500.0,100000.0,50
19237,47,48,36.0,47,48.0,60,239000.0,500.0,110000.0,48


In [226]:
data_22=newff_22.copy()
data_22

,overall,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power
0,92,94,91.0,96,95.0,93,144300000.0,320000.0,78000000.0,86
1,92,93,79.0,88,86.0,92,197200000.0,270000.0,119500000.0,90
2,91,94,80.0,95,88.0,91,83300000.0,270000.0,45000000.0,94
3,91,89,86.0,93,94.0,91,238700000.0,270000.0,129000000.0,80
4,91,91,93.0,89,88.0,91,232200000.0,350000.0,125500000.0,91
...,...,...,...,...,...,...,...,...,...,...
19234,47,53,46.0,37,48.0,52,114000.0,1000.0,70000.0,46
19235,47,49,50.0,47,46.0,59,193000.0,500.0,110000.0,49
19236,47,46,45.0,36,49.0,55,175000.0,500.0,100000.0,50
19237,47,48,36.0,47,48.0,60,239000.0,500.0,110000.0,48


In [227]:
y=newff_22['overall']
X=newff_22[[ 'movement_reactions', 'passing', 'mentality_composure',
       'dribbling', 'potential', 'release_clause_eur', 'wage_eur', 'value_eur',
       'power_shot_power']]

In [228]:
sc=StandardScaler()
scaled=sc.fit_transform(X)
X=pd.DataFrame(scaled, columns=X.columns)


In [229]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [230]:
from sklearn.ensemble import RandomForestRegressor
rf_22 = RandomForestRegressor()
rf_22.fit(Xtrain, Ytrain)

RandomForestRegressor()

In [231]:
y_22=rf_22.predict(Xtest)
y_22

array([62.  , 61.99, 64.33, ..., 61.05, 59.74, 64.71])

In [232]:
mean_absolute_error(y_22,Ytest)

0.6902676715176715

In [233]:
dt_22 = DecisionTreeRegressor(random_state=42)
dt_22.fit(Xtrain, Ytrain)

DecisionTreeRegressor(random_state=42)

In [234]:
y_22 = dt_22.predict(Xtest)
y_22

array([61., 62., 64., ..., 61., 60., 66.])

In [235]:
mean_absolute_error(y_22,Ytest)

0.8071725571725572

In [245]:
xgb_22 = XGBRegressor()
xgb_22.fit(Xtrain, Ytrain)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [246]:
xgb_22

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [247]:
xgb_22_prediction = xgb_22.predict(Xtest)

In [238]:
mean_absolute_error(xgb_22,Ytest)

0.7942163314740028

In [239]:
#finetune decisiontree for fifa22
param_distributions = {
    'max_depth': np.arange(2, 10, step=1),
    'min_samples_split': np.arange(2, 30, step=10),
    'max_features': np.arange(1, 12, step=1)
}
n_iter = 30
random_search_22 = RandomizedSearchCV(
    estimator=DecisionTreeRegressor(),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=5
)
random_search_22.fit(Xtrain, Ytrain)


RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_iter=30,
                   param_distributions={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                                        'max_features': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
                                        'min_samples_split': array([ 2, 12, 22])})

In [240]:
best_params = random_search_22.best_params_
best22_dt = DecisionTreeRegressor(**best_params)
best22_dt.fit(Xtrain, Ytrain)

DecisionTreeRegressor(max_depth=9, max_features=10, min_samples_split=22)

In [241]:
y_pred_22 = best_dt.predict(Xtest)
mean_absolute_error(Ytest, y_pred_22)

1.9547838835615918

saving model as pickle file

In [248]:
import pickle
import joblib

joblib.dump(xgb_22, 'xgb_22.pkl')

model_saved='/content/drive/MyDrive/Colab Notebooks/xgb_22.pkl'
with open(model_saved,'wb')as file:
  pickle.dump(xgb_22,file)

In [249]:
xgb_22


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)